Импортируем и декларируем всякие нужные штуки

In [80]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load("en_core_web_sm")
sw = stopwords.words("english")
session = requests.session()
ua = UserAgent()
headers = {"User-Agent": ua.random}

Пишем функцию получения отзывов. Я выбрала для краулинга сайт Tripadvisor и отзывы на некоторые кафешки в Нью-Йорке (попутешествуем хотя бы так...)

In [4]:
def get_reviews(url):
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.text, "html.parser")
    reviews = []
    review_elements = soup.find_all("div", class_= "review-container")
    for review in review_elements:
        rating = review.find("span", class_= "ui_bubble_rating")["class"][1].split("_")[-1]
        content = review.find("p", class_= "partial_entry").text
        reviews.append({"rating": rating, "content": content})
    return reviews

Получаем сначала отзывы на одно популярное и любимое людьми место, откуда скорее всего легко будет повытаскивать положительные отзывы



In [7]:
url = "https://www.tripadvisor.com/Restaurant_Review-g60763-d11916965-Reviews-Non_Solo_Piada-New_York_City_New_York.html"
good_reviews = []

for page in range(10):
    page_url = f'{url.split("-Reviews-")[0]}-or{page*10}-{url.split("-Reviews-")[1]}'
    page_reviews = get_reviews(page_url)
    good_reviews.extend(page_reviews)
    if len(good_reviews) >= 100:
        break

Делаем список собственно самих положительных отзывов (только с оценкой 5/5).

In [ ]:
pos = []
for review in good_reviews:
    if review["rating"] == "50" and review["content"] not in pos:
        pos.append(review["content"])
pos = pos[:50]
#for review in pos:
    #print(review)
   #print("\n")

То же самое для отрицательных:

In [18]:
url = "https://www.tripadvisor.com/Restaurant_Review-g60763-d424088-Reviews-Nello-New_York_City_New_York.html"
bad_reviews = []
for page in range(25):
    page_url = f'{url.split("-Reviews-")[0]}-or{page*10}-{url.split("-Reviews-")[1]}'
    page_reviews = get_reviews(page_url)
    bad_reviews.extend(page_reviews)


In [ ]:
neg = []
for review in bad_reviews:
    if review["rating"] == "10" and review["content"] not in neg:
        neg.append(review["content"])
neg = neg[:50]
#for review in neg:
    #print(review)
    #print("\n")

Функция для приведения к начальной форме, нижнему регистру и фильтра стоп-слов

In [22]:
def lemmetize(words):
    lemmed_words = []
    words = [w.lower() for w in word_tokenize(words) if w.isalpha()]
    tokens = " ".join([w for w in words if w not in sw])
    doc = nlp(tokens)
    for token in doc:
        lemmed_words.append(token.lemma_)
    lemmed_text = " ".join(lemmed_words)
    return(lemmed_text)

Проделываем это всё с обоими наборами отзывов

In [25]:
lemma_pos = []
lemma_neg = []
for p in pos:
    lemma_pos.append(lemmetize(p))
for n in neg:
    lemma_neg.append(lemmetize(n))

In [26]:
negj = " ".join(lemma_neg).split()
posj = " ".join(lemma_pos).split()

Подсчитываем слова в отзывах

In [27]:
posdict = {}
negdict = {}
for w in posj:
    if w not in posdict:
        posdict[w] = 1
    else:
        posdict[w] += 1
for w in negj:
    if w not in negdict:
        negdict[w] = 1
    else:
        negdict[w] += 1

Выделяем ключевые слова, которые либо встречались только в отзывах одной тональности, либо встречались в отзывах одной тональности чаще, чем в отзывах другой тональности (это мой способ решения проблемы пустых множеств), и присваиваем им "вес" разницу квхождений в оложительные и в отрицательные отзывы (в зависимости от того, чего больше).

In [87]:
keyneg = {}
keypos = {}
for p in posdict:
    if p in negdict:
        if posdict[p] > negdict[p]:
            score = posdict[p] - negdict[p]
    else:
        score = posdict[p]
    keypos[p] = score
for n in negdict:
    if n in posdict:
        if negdict[n] > posdict[n]:
            score = negdict[n] - posdict[n]
    else:
        score = negdict[n]
    keyneg[n] = score

Набираем отзывы для проверки

In [29]:
url = "https://www.tripadvisor.com/Restaurant_Review-g60763-d3491934-Reviews-The_Halal_Guys-New_York_City_New_York.html"
test = []
for page in range(90):
    page_url = f'{url.split("-Reviews-")[0]}-or{page*10}-{url.split("-Reviews-")[1]}'
    page_reviews = get_reviews(page_url)
    test.extend(page_reviews)

In [30]:
testneg = []
for review in test:
    if review["rating"] == "10" and review["content"] not in testneg:
        testneg.append(review["content"])
len(testneg)

30

In [31]:
testpos = []
for review in test:
    if review["rating"] == "50" and review["content"] not in testpos:
        testpos.append(review["content"])
    if len(testpos) == 30:
        break

In [32]:
ans = {}
for i in testneg:
    ans[i] = 0
for i in testpos:
    ans[i] = 1

Функция для определения тональности: каждое слово в отзыве приводим к нормальному виду, далее смотрим, является ли оно ключевым для положительной и отрицательной тональности. Прибавляем к его "счету" то число, которое соответствует ключевому слову.

In [105]:
def sentiment(keypos, keyneg, review):
    pos_score = 0
    neg_score = 0
    lem_review = lemmetize(review)
    for word in lem_review:
        if word in keypos:
            pos_score += keypos[word]
        if word in keyneg:
            neg_score += keyneg[word]
    if pos_score > neg_score:
        return 1
    else:
        return 0

Проверяем аккураси: если правильно определена тональность, добавляем балл. Потом делим сумму баллов на длину тестовой выборки

In [42]:
def accuracy(ans, pred):
    sum = 0
    for i in pred:
        if pred[i] == ans[i]:
            sum += 1
    return sum/len(ans)

In [109]:
pred = {}
for i in ans:
    pred[i] = sentiment(keypos, keyneg, i)

In [112]:
accuracy(ans, pred)

0.4666666666666667

Получилось, конечно, так себе. Отсюда переходим к возможным улучшениям этой убогой штуки:

1. создать и применить топико-специфичный список стоп-слов. Для отзывов на рестораны, например, там будут слова типа "кафе, еда, персонал" и т.д. Создать такой список можно на основе большой выборки отзывов, выделив те слова, которые встречаются в большинстве из них. Однозначно тогда будет меньше негативных и позитивных маркеров, не имеющих в своей сути ничего негативного или позитивного

2. отзывы бывают длинные и короткие. Можно попробовать давать больший вес словам в маленьких отзывах и меньший в больших. Логика за этим такая: в маленьком отзыве скорее всё лаконичнее и слова напрямую описывают эмоциональную реакцию рецензента, в большом же может быть много воды и каких-то витиеватых, детальных сравнений/описаний/рассуждений (а соответственно и лишних слов, которые автор написал просто от балды). это не точно, но можно попробовать

3. можно попробовать учитывать отрицания перед словами и при наличии такового трактовать слово противоположно либо рассматривать его с отрицанием как одно целое, например, если встретили not good в отрицательном отзыве, то не бежим записывать good в отрицательный список, а либо наоборот тащим его в отрицательный (not меняет значение на противоположное => слово на самом деле имеет положительную коннотацию), либо тащим в отрицательный список словосочетание not good

надеюсь, что-то из этого засчитается, как улучшение.... thanks for coming to my ted talk

<img src="https://i.pinimg.com/originals/1c/b6/04/1cb604515b145c012a82f7ce6ed75e0f.jpg">
